In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
from blackbox_wrapper import BlackboxWrapper
import keras
from datasets import build_cbf
from variational_autoencoder import load_model
from neighborhood_generators import NeighborhoodGenerator
from lore_wrapper import LoreWrapper
from utils import choose_z
from joblib import load
from lasts import load_multiple_lasts
import time
import numpy as np
from experiments_neighborhoods_surrogates import multiple_tests
from sbgdt import Sbgdt, generate_n_shapelets_per_size
from saxdt import Saxdt

Using TensorFlow backend.
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

In [3]:
random_state = 0
np.random.seed(random_state)
dataset_name = "cbf"
blackbox_name = "resnet"

file_path = parentdir + "/saved/" + dataset_name + "_" + blackbox_name + "_" + time.strftime("%Y%m%d_%H%M%S")

(X_train, y_train, X_val, y_val,
 X_test, y_test, X_exp_train, y_exp_train,
 X_exp_val, y_exp_val, X_exp_test, y_exp_test) = build_cbf(n_samples=600, random_state=random_state)

knn = load(parentdir + "/trained_models/cbf/cbf_knn.joblib")
resnet = keras.models.load_model(parentdir + "/trained_models/cbf/cbf_resnet.h5")

_, _, autoencoder = load_model(parentdir + "/trained_models/cbf/cbf_vae")

blackbox = BlackboxWrapper(knn, 2, 1)
blackbox = BlackboxWrapper(resnet, 3, 2)
encoder = autoencoder.layers[2]
decoder = autoencoder.layers[3]

DATASET INFO:
X SHAPE:  (600, 128, 1)
y SHAPE:  (600,)

CLASSES BALANCE
0 :  0.33
1 :  0.33
2 :  0.33

SHAPES:
BLACKBOX TRAINING SET:  (268, 128, 1)
BLACKBOX VALIDATION SET:  (68, 128, 1)
BLACKBOX TEST SET:  (84, 128, 1)
EXPLANATION TRAINING SET:  (115, 128, 1)
EXPLANATION VALIDATION SET:  (29, 128, 1)
EXPLANATION TEST SET:  (36, 128, 1)
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
neighborhood_names = ["gaussian_matched", "uniform_sphere", "gaussian", "matched_uniform", "lore"]
neighborhood_list = [NeighborhoodGenerator, NeighborhoodGenerator, NeighborhoodGenerator, NeighborhoodGenerator, LoreWrapper]
neighborhood_kwargs_list = [
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian_matched",
            "sampling_kind": "gaussian_matched",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": False
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "uniform_sphere",
            "sampling_kind": "uniform_sphere",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": True
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian",
            "sampling_kind": "gaussian",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": False
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian_matched",
            "sampling_kind": "uniform_sphere",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": True
        },
        {"verbose": False, "samples": 500, "ngen": 10}
    ]

In [6]:
n_shapelets_per_size = generate_n_shapelets_per_size(X_train.shape[1], n_shapelets_per_length=4, 
                                                     min_length=8, start_divider=2, divider_multiplier=1.2)
surrogate_kwargs_list = [{"shapelet_model_params": {"max_iter": 50, "n_shapelets_per_size":n_shapelets_per_size}, 
                          "random_state": random_state},
                             {"random_state": np.random.seed(0), "create_plotting_dictionaries":False}]

surrogate_list = [Sbgdt, Saxdt]
surrogate_names = ["stdt", "saxdt"]

In [7]:
%%time
out = multiple_tests(
        X_exp_test,
        blackbox,
        encoder,
        decoder,
        neighborhood_list=neighborhood_list,
        neighborhood_kwargs_list=neighborhood_kwargs_list,
        neighborhood_names=neighborhood_names,
        surrogate_list=surrogate_list,
        surrogate_kwargs_list=surrogate_kwargs_list,
        surrogate_names=surrogate_names,
        file_path=file_path,
        random_state=random_state,
        n_instances_to_explain=None,
        verbose=True,
        simple_dump=[False, True],
        labels=["cylinder", "bell", "funnel"]
    )

--- MULTIPLE LASTS TESTS ---

 --> RECONSTRUCTION ACCURACY: 1.0


--- NEIGHBORHOOD GENERATIONS ---

 --> LASTS_LORE
  NEIGHGEN: 1 / 1 | INSTANCE: 1 / 36
calculating feature values
gen	nevals	avg     	min     	max     
0  	250   	0.496654	0.496654	0.496654
1  	216   	0.527814	0.0066974	0.993307


KeyboardInterrupt: 